In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data').get('EdbName')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def main():
    # List of indicator IDs you want to fetch
    indicator_ids = ['W8864032','C2505201312426221','GL92-SIN-CRK','C2505201305599445',
                     'GO10SGCKMc1', 'C2505201300158987', 'C2505201241250955','C2505231634335044',
                     'W8864033','C2307266390','C2302232479','C2302234358','C2505201308049486','C2505281711522529'
                     ]


    # Dictionary to store DataFrames for each indicator
    data_frames = {}

    for indicator_id in indicator_ids:
        data = fetch_indicator_details(indicator_id)
        if data:
            # Create a DataFrame with DataTime as index
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            # Only keep the 'Value' column and rename it to the indicator ID
            df = df[['Value']].rename(columns={'Value': fetch_indicator_name(indicator_id)})
            data_frames[indicator_id] = df

    # Concatenate all DataFrames along the columns
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print(result_df.info())
        result_df.to_excel("data_input/中国汽柴油出口计划量.xlsx")
        print("Data saved successfully as '中国汽柴油出口计划量.xlsx'")



if __name__ == "__main__":
    main()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3732 entries, 2011-04-19 to 2025-10-30
Data columns (total 14 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   中国汽油出口计划量                                    39 non-null     float64
 1   汽油：产出率：中国：主营炼厂（周）/4WMA同差                     203 non-null    float64
 2   新加坡92#汽油裂解价差                                 3642 non-null   float64
 3   中国汽油产量同差                                     238 non-null    float64
 4   新加坡10ppm柴油裂解                                 3016 non-null   float64
 5   柴油：主营炼厂：产量：中国（周）同差                           203 non-null    float64
 6   中国柴油库存4周环差                                   328 non-null    float64
 7   （中国独立炼厂柴油库存+中国主营成品油销售公司柴油库存）/中国柴油实际消费量/4WMA  286 non-null    float64
 8   中国柴油出口计划量                                    39 non-null     float64
 9   汽油出口利润-柴油出口利润                                2032 non-nu

Data saved successfully as '中国汽柴油出口计划量.xlsx'
